# 1️⃣ Re-Imaging Price Trends - 이미지 생성

**목적**: 주가 데이터를 캔들스틱 차트 이미지로 변환하여 디스크에 저장

**완료 후**: `2_model_training.ipynb` 실행

In [ ]:
# 환경 설정 및 최적화 확인
!pip install -r requirements.txt

from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir('/content/drive/MyDrive/ReImaging_Price_Trends')
print(f"📁 현재 디렉토리: {os.getcwd()}")
print(f"📄 파일 목록: {[f for f in os.listdir('.') if not f.startswith('.')]}")

# Numba JIT 성능 최적화 확인
try:
    import numba
    print(f"✅ Numba JIT 사용 가능: {numba.__version__}")
    print("   🚀 이미지 생성 속도가 50-100배 향상됩니다!")
except ImportError:
    print("❌ Numba 설치 실패 - requirements.txt 확인 필요")

# 메모리 상태 확인
import psutil
memory = psutil.virtual_memory()
print(f"💾 사용 가능한 메모리: {memory.available // (1024**3):.1f}GB")
if memory.available < 2 * (1024**3):  # 2GB 미만
    print("⚠️  메모리 부족 시 --parallel 1 옵션 사용 권장")

In [ ]:
# 데이터 파일 확인
data_files = [
    'data/data_1993_2000_train_val.parquet',
    'data/data_2001_2019_test.parquet'
]

print("📊 데이터 파일 확인:")
all_exist = True
for file in data_files:
    if os.path.exists(file):
        size_mb = os.path.getsize(file) / (1024**2)
        print(f"✅ {file} ({size_mb:.1f}MB)")
    else:
        print(f"❌ {file} 없음")
        all_exist = False

if not all_exist:
    print("\n⚠️ 데이터 파일이 없습니다. data/datageneration.ipynb를 먼저 실행하세요.")

In [ ]:
# 🚀 논문 저자 원본 형식 이미지 생성 (.dat + .feather)
print("🚀 원본 형식 이미지 생성 시작...")

# 5일 이미지 (원본 형식)
print("\n1️⃣ 5일 이미지 (원본 형식)")
!python create_original_format.py --image_days 5 --mode train --sample_rate 1.0
!python create_original_format.py --image_days 5 --mode test --sample_rate 1.0

# 20일 이미지 (원본 형식)
print("\n2️⃣ 20일 이미지 (원본 형식)")
!python create_original_format.py --image_days 20 --mode train --sample_rate 1.0
!python create_original_format.py --image_days 20 --mode test --sample_rate 1.0

# 60일 이미지 (원본 형식)
print("\n3️⃣ 60일 이미지 (원본 형식)")
!python create_original_format.py --image_days 60 --mode train --sample_rate 1.0
!python create_original_format.py --image_days 60 --mode test --sample_rate 1.0

print("\n✅ 모든 원본 형식 이미지 생성 완료!")
print("📊 저장 형식: .dat (이미지) + .feather (라벨)")

In [ ]:
# 생성된 원본 형식 이미지 확인
import pandas as pd
import os
import numpy as np

print("📊 생성된 원본 형식 이미지 요약:")

# 원본 형식 디렉토리 확인
original_dirs = {
    'weekly_5d': (5, '5d_week_has_vb_[5]_ma'),
    'monthly_20d': (20, '20d_month_has_vb_[20]_ma'), 
    'quarterly_60d': (60, '60d_quarter_has_vb_[60]_ma')
}

base_dir = 'img_data_reconstructed'
if os.path.exists(base_dir):
    total_images = 0
    total_size_gb = 0
    success_count = 0
    
    for dir_name, (win_size, prefix) in original_dirs.items():
        img_dir = os.path.join(base_dir, dir_name)
        
        if os.path.exists(img_dir):
            # .dat 및 .feather 파일 확인
            dat_files = [f for f in os.listdir(img_dir) if f.endswith('.dat')]
            feather_files = [f for f in os.listdir(img_dir) if f.endswith('.feather')]
            
            dir_images = 0
            dir_size = 0
            
            # 각 연도별 파일 확인
            for dat_file in dat_files:
                dat_path = os.path.join(img_dir, dat_file)
                
                # 이미지 개수 계산 (.dat 파일 크기 기반)
                file_size = os.path.getsize(dat_path)
                if win_size == 5:
                    image_size = 32 * 15  # 5일: 32x15
                elif win_size == 20:
                    image_size = 64 * 60  # 20일: 64x60  
                else:  # 60
                    image_size = 96 * 180  # 60일: 96x180
                
                num_images = file_size // image_size
                dir_images += num_images
                dir_size += file_size
            
            # .feather 파일 크기 추가
            for feather_file in feather_files:
                feather_path = os.path.join(img_dir, feather_file)
                dir_size += os.path.getsize(feather_path)
            
            size_gb = dir_size / (1024**3)
            total_size_gb += size_gb
            
            print(f"✅ {dir_name}: {dir_images:,}개 이미지, {size_gb:.2f}GB")
            print(f"   .dat 파일: {len(dat_files)}개, .feather 파일: {len(feather_files)}개")
            
            total_images += dir_images
            success_count += 1
        else:
            print(f"❌ {dir_name}: 생성되지 않음")
    
    print(f"\n📈 최종 결과:")
    print(f"   성공: {success_count}/{len(original_dirs)} 디렉토리")
    print(f"   총 이미지: {total_images:,}개")
    print(f"   총 용량: {total_size_gb:.2f}GB")
    print(f"   이미지당 평균 용량: {total_size_gb*1024*1024/max(total_images,1):.1f}KB")
    
    if success_count == len(original_dirs):
        print(f"\n🎉 모든 원본 형식 이미지 생성 완료!")
        print(f"📁 저장 경로: {base_dir}/")
        print(f"📊 형식: .dat (바이너리 이미지) + .feather (라벨)")
        print(f"\n➡️  논문 저자 원본과 동일한 형식으로 저장됨")
    else:
        print(f"\n⚠️  {len(original_dirs)-success_count}개 디렉토리 생성 실패")
        print("   create_original_format.py 오류 로그를 확인하세요.")
        
    # label_columns.txt 파일 확인
    label_file = os.path.join(base_dir, 'label_columns.txt')
    if os.path.exists(label_file):
        print(f"\n📋 메타데이터: {label_file} 생성됨")
        with open(label_file, 'r') as f:
            print("   내용:")
            for line in f.readlines()[:3]:  # 처음 3줄만 표시
                print(f"   {line.strip()}")
else:
    print(f"❌ {base_dir} 디렉토리가 생성되지 않았습니다.")
    print("   create_original_format.py 실행을 확인하세요.")